In [1]:
import numpy as np
import os
import pandas as pd
from skimage.io import imread, imsave
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import svm
import torchvision.transforms as T
import torch
torch.manual_seed(42)

import warnings
warnings.filterwarnings('ignore')

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
er_trdat_path = './data/train/ER/'
nr_trdat_path = './data/train/NR/'
cropdat_path = './data/train/all_cropped/'
balanced_path = './data/train/all_balanced/'

### Crop Images

In [3]:
%%time
counts = dict()
i = 0
for filename in os.listdir(er_trdat_path):
    fn_parts = filename.split('_')
    ind =  fn_parts.index('ER')
    regr = fn_parts[ind+1]
    if regr not in {'1', '3', '6', '10', '20', '30'}:
        print(f'error fn_parts[7]')
        print(fn_parts)
        print(filename)
        break    
    img = imread(er_trdat_path + filename)    
    img = img[192:384, 192:384]
    img = resize(img, (64, 64))
    img = img*255
    img = img.astype(np.uint8)
    imsave(cropdat_path + f'{i}-ER-{regr}.png', img)
    counts[f'ER-{regr}'] = counts.get(f'ER-{regr}', 0) + 1
    i += 1
for filename in os.listdir(nr_trdat_path):
    fn_parts = filename.split('_')
    ind =  fn_parts.index('NR')
    regr = fn_parts[ind+1]
    if regr not in {'1', '3', '6', '10', '20', '30'}:
        print(f'error fn_parts[7]')
        print(fn_parts)
        print(filename)
        break    
    img = imread(nr_trdat_path + filename)
    img = img[192:384, 192:384]
    img = resize(img, (64, 64))
    img = img*255
    img = img.astype(np.uint8)
    imsave(cropdat_path + f'{i}-NR-{regr}.png', img)
    counts[f'NR-{regr}'] = counts.get(f'NR-{regr}', 0) + 1
    i += 1

Wall time: 59.6 s


### balance classes

In [ ]:
transforms = torch.nn.Sequential(
    T.RandomAffine(degrees=45),
    T.RandomHorizontalFlip(p=0.5),
    T.GaussianBlur(3, sigma=(0.1, 2.0)),
)

In [ ]:
for filename in os.listdir('./data/train/small/'):
    img = imread('./data/train/small/' + filename) 
    fn_parts = filename.split('_')
    ind =  fn_parts.index('ER')
    regr = fn_parts[ind+1]       
    img = img[192:384, 192:384]
    img = resize(img, (64, 64))
    img = img*255
    img = img.astype(np.uint8)
    imsave(balanced_path + f'{i}-ER-{regr}.png', img)
    i += 1
    if regr in {'1', '6', '20'}:
        for

In [ ]:
%%time
i = 0
for filename in os.listdir(er_trdat_path):
    fn_parts = filename.split('_')
    ind =  fn_parts.index('ER')
    regr = fn_parts[ind+1]
    if regr not in {'1', '3', '6', '10', '20', '30'}:
        print(f'error fn_parts[7]')
        print(fn_parts)
        print(filename)
        break    
    img = imread(er_trdat_path + filename)    
    img = img[192:384, 192:384]
    img = resize(img, (64, 64))
    img = img*255
    img = img.astype(np.uint8)
    imsave(cropdat_path + f'{i}-ER-{regr}.png', img)
    i += 1
for filename in os.listdir(nr_trdat_path):
    fn_parts = filename.split('_')
    ind =  fn_parts.index('NR')
    regr = fn_parts[ind+1]
    if regr not in {'1', '3', '6', '10', '20', '30'}:
        print(f'error fn_parts[7]')
        print(fn_parts)
        print(filename)
        break    
    img = imread(nr_trdat_path + filename)
    img = img[192:384, 192:384]
    img = resize(img, (64, 64))
    img = img*255
    img = img.astype(np.uint8)
    imsave(cropdat_path + f'{i}-NR-{regr}.png', img)
    i += 1